In [ ]:
# 参考 https://colab.research.google.com/github/TadaoYamaoka/ShogiAIBook2/blob/main/notebook/train.ipynb#scrollTo=zAT8jrKysnZS
# dlshogiの評価関数モデルをCore ML形式に変換
# バッチサイズNのモデルを出力する。Nは定数。
# Nを変えたモデルをいくつか出力。

In [ ]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime
!date

Sat Jan 29 08:47:38 JST 2022


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -l "/content/drive/My Drive/ShogiAIBookData/"

total 9932211
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-001.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-002.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-003.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-004.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-005.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-006.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-007.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-008.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-009.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-010.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-011.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-012.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-013.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlsh

In [ ]:
!pip install dlshogi==0.1.1

     |████████████████████████████████| 3.3 MB 6.1 MB/s 


In [ ]:
!pip install coremltools

     |████████████████████████████████| 1.6 MB 5.0 MB/s 


In [ ]:
%cd /content/ShogiAIBook

[Errno 2] No such file or directory: '/content/ShogiAIBook'
/content


In [ ]:
!ln -s "/content/drive/My Drive/ShogiAIBook/log"   ./log
!ln -s "/content/drive/My Drive/ShogiAIBook/model" ./model
!ln -s "/content/drive/My Drive/ShogiAIBookData"   ./data

In [ ]:
!cp ./data/model/model_resnet10_swish-072 ./model/

In [ ]:
import torch
from dlshogi.common import *
from dlshogi.network.policy_value_network import policy_value_network
from dlshogi import serializers
from dlshogi import cppshogi

In [ ]:
model = policy_value_network("resnet10_swish", add_sigmoid=True)

In [ ]:
model.set_swish(False)#swishをx*sigmoid(x)で計算するモード

In [ ]:
serializers.load_npz("./model/model_resnet10_swish-072", model, False)

In [ ]:
model.eval()

PolicyValueNetworkAddSigmoid(
  (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l10): Conv2d(192, 192, k

In [ ]:
class SingleInputModel(torch.nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model = model
  def forward(self, x):
    return self.model(x[:, 0:62, :, :], x[:, 62:119, :, :])

In [ ]:
si_model = SingleInputModel(model)
si_model.eval()

SingleInputModel(
  (model): PolicyValueNetworkAddSigmoid(
    (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
import coremltools as ct

In [ ]:
# 2^n、2^n-1, 2^n-1
batch_sizes = [1, 4, 16, 64, 256, 1024, 3, 5, 15, 17, 63, 65, 255, 257, 1023, 1025]

In [ ]:
bs_dirs = " ".join(f"./model/batch{bs}" for bs in batch_sizes)

In [ ]:
!mkdir -p {bs_dirs}

In [ ]:
for batch_size in batch_sizes:
  traced_model = torch.jit.trace(si_model, torch.zeros(batch_size, 119, 9, 9))
  ct_input_x = ct.TensorType(name='x', shape=(batch_size, 119, 9, 9))
  mlmodel = ct.convert(traced_model, inputs=[ct_input_x])
  # このままだと、出力変数の名前がvar_577のような実装依存の値になり、Swift側での受け取りに不便なので名前を固定する
  spec = mlmodel.get_spec()
  move_name, result_name = mlmodel.output_description
  ct.utils.rename_feature(spec, move_name, "move")
  ct.utils.rename_feature(spec, result_name, "result")
  mlmodel = ct.models.MLModel(spec)
  mlmodel.save(f"./model/batch{batch_size}/DlShogiResnet10SwishBatch.mlmodel")

Running MIL Common passes:   0%|          | 0/34 [00:00<?, ? passes/s]/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '406', of the source model, has been renamed to 'var_406' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '429', of the source model, has been renamed to 'var_429' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 257/257 [00:01<00:00, 255.57 ops/s]
